In [25]:
%load_ext ipyslurm
%slogin login-1instructor-011

The ipyslurm extension is already loaded. To reload it, use:
  %reload_ext ipyslurm
Logging in to instructor-011@login-1
Password:········


Logged in to login-1

In [31]:
%%sbatch
#!/usr/bin/env bash
#SBATCH --job-name=Rmpi
#SBATCH --nodes=3
#SBATCH --time=00:02:00     
#SBATCH --ntasks-per-node=6
#SBATCH --cpus-per-task=1
#SBATCH --output=rmpi-%j.log
#SBATCH --partition=nu-wide

cd $SLURM_SUBMIT_DIR

module load mpich/3.2.1-gcc-7.2.0
module load gcc/7.2.0
module load R/3.5.1
module list
TOOLS=/home/instructor-011/tools/miniconda3/envs/SIMMAC/bin

WORKERS=$(($SLURM_NTASKS-1))

HOSTFILE=$(pwd)/.host.$SLURM_JOBID
#echo  $SLURM_NODEFILE > ${HOSTFILE}
srun hostname -s &> $HOSTFILE

# Create test R script
cat > test.R << EOF
# Load library
library(Rmpi)

# In case R exits unexpectedly, have it automatically clean up
# resources taken up by Rmpi (slaves, memory, etc...)
.Last <- function(){
   if (is.loaded("mpi_initialize")){
     if (mpi.comm.size(1) > 0){
       print("Please use mpi.close.Rslaves() to close workers.")
       mpi.close.Rslaves()
     }
     print("Please use mpi.quit() to quit R")
     .Call("mpi_finalize")
   }
}

# Spawn workers
mpi.spawn.Rslaves(nslaves=$WORKERS)

# Execute a command on each worker
mpi.remote.exec(paste("I am",mpi.comm.rank(),"of",mpi.comm.size()), ret=FALSE)
#mpi.remote.exec(sum(1:mpi.comm.rank()))
mpi.close.Rslaves()
mpi.quit()
EOF

echo "Using $SLURM_JOB_NUM_NODES nodes"
mpiexec -n $SLURM_JOB_NUM_NODES Rscript test.R
#mpirun -np $SLURM_NTASKS $TOOLS/Rscript test.R

Submitted batch job 4583


In [33]:
%%sbash
squeue -u instructor-011

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
